In [ ]:
from vsa.vsa import VSA
model = "SOFTWARE"
test_dir = f"tests/test_{model}"

vsa = VSA(
    root=test_dir,
    dim=10,
    model=model,
    num_factors=3,
    num_codevectors=3
)



In [ ]:
labels, vectors = vsa.sample(5)
emp = vsa.empty(2, 10)
emp, vectors

In [ ]:
vsa.bind(vectors[0], vectors[1]), vsa.bundle(vectors[0], vectors[1])

In [ ]:
vsa.multibind(vectors), vsa.multiset(vectors, False)

In [ ]:
vsa.similarity(vectors[0], vectors[1])

In [ ]:
import torch
weights = torch.tensor([1,2,3,4,-1])
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))
# weights * vectors.transpose(-2,-1)

In [ ]:
weights = torch.tensor([[1,2,3,4,-1], [-1,-2,-3, -4, 1]])
vectors.unsqueeze(0).repeat(2,1,1)
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))

In [3]:
from main import *
from vsa.vsa import VSA

VSA_MODEL = 'SOFTWARE' # 'SOFTWARE', 'HARDWARE'

# Default values
DIM = 2000
FACTORS = 4
CODEVECTORS = 3
# CODEVECTORS : tuple = (3,3,3,10) 
NOISE_LEVEL = 0.0   # compositional vector noise
ITERATIONS = 100    # max number of iterations for factorization
NORMALIZE = False    # normalize the initial estimate and the input vector (when the input is a bundled vector)
ACTIVATION = 'NONE' # 'NONE', 'ABS', 'NONNEG'
RESONATOR_TYPE = "SEQUENTIAL" # "CONCURRENT", "SEQUENTIAL"
ARGMAX_ABS = True

test_dir = f"tests/{VSA_MODEL}-{DIM}d-{FACTORS}f-{v_name(CODEVECTORS)}"

vsa = VSA(
    root=test_dir,
    model=VSA_MODEL,
    dim=DIM,
    num_factors=FACTORS,
    num_codevectors=CODEVECTORS,
    seed = 0
)

resonator_network = Resonator(vsa, type=RESONATOR_TYPE, activation=ACTIVATION, iterations=ITERATIONS, argmax_abs=ARGMAX_ABS)
init_estimates = gen_init_estimates(vsa, NORMALIZE, 1)

label = [(1, 2, 0, 1)]
input = vsa[label]

outcomes, convergence = resonator_network(input, init_estimates)
outcome = outcomes[0]

if (outcome not in label):
    print(Fore.RED + f"Test Failed: " + f"Label = {label}, Outcome = {outcome}" + Fore.RESET + f"    Convergence: {convergence}")
else:
    print(f"Test Passed: " + f"Label = {label}, Outcome = {outcome}    Convergence: {convergence}")

Test Passed: Label = [(1, 2, 0, 1)], Outcome = (1, 2, 0, 1)    Convergence: 1
